In [1]:
import comm as cmm
import codecs

def GenCreateTable(df):

    tbNames = df["TableName"].unique()
    for tb in tbNames:
        colsStr, ColExtFormats="",""
        dfs = df[df["TableName"]==tb]
      
        for e in dfs["colComp"]:
            colsStr += e +"\n"

        for e in dfs["colExt"]:
            ColExtFormats += e +"\n"

        tmpFile = codecs.open("template/createTable.sql","r").read().format(tb=tb, colsStr=colsStr, ColExtFormats=ColExtFormats)
        f = codecs.open(f"tables/tb{tb}.sql", "w","utf-8")
        f.write(tmpFile)
        f.close()


def GenOneDeleteSP(tb, dfc):
    pt_input, pt_Declare, pt_SetValue = "","",""

    dft = df_sp[df_sp["tableName"]==tb]
    tbDscr = dft["tbDscr"].values[0]         #取得單一值
    dfc = dfc[dfc["tableName"]==tb]    
    dfc = dfc.loc[dfc["col"].isin(["SeqNo"])]

    for e in dfc:
        d = dfc[e].values[0]
        pt_input += f"{d}\n\t" if (e=="cmmt_InputParas") else ""
        pt_Declare += f"{d}\n\t" if (e=="cmmt_Declares") else ""
        pt_SetValue += f"{d}\n\t" if (e=="cmmt_SetValue") else ""
    
    pt_DateTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    tmpFile = codecs.open("template/agdSP.Delete.sql","r",encoding="utf-8").read().format(tb=tb, tbDscr=tbDscr, pt_DateTime=pt_DateTime, pt_input=pt_input[:-2], pt_Declare=pt_Declare[:-2], pt_SetValue=pt_SetValue[:-2])
    
    f = codecs.open(f"agdSP/agdSP.usp{tb}.Delete.sql", "w","utf-8")
    f.write(tmpFile)
    f.close()
    print(f)


def GenOneGetSP(tb, dfc):
    pt_input, pt_Declare, pt_SetValue = "","",""
    pt_select, pt_fCol ="",""

    dft = df_sp[df_sp["tableName"]==tb]
    tbDscr = dft["tbDscr"].values[0]         #取得單一值
    dfc = dfc[dfc["tableName"]==tb]    

    for e in dfc.loc[dfc["col"].isin(["SeqNo"])]:
        d = dfc[e].values[0]
        pt_input += f"{d}\n\t" if (e=="cmmt_InputParas") else ""
        pt_Declare += f"{d}\n\t" if (e=="cmmt_Declares") else ""
        pt_SetValue += f"{d}\n\t" if (e=="cmmt_SetValue") else ""

    
    aa = dfc["cmmt_select"].tolist()
    bb = dfc["fCol"].tolist()

    pt_select = "".join([f"{x}\n" for x in aa])
    pt_fCol   = "".join(f"{x}\n\t\t\t" for x in bb)

    tmpFile = codecs.open("template/agdSP.Get.sql",mode="r",encoding="utf-8",errors='strict', buffering=-1).read().format(tb=tb, tbDscr=tbDscr, pt_DateTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), pt_input=pt_input[:-2], pt_Declare=pt_Declare[:-2], pt_SetValue=pt_SetValue[:-2], pt_select = pt_select[:-1], pt_fCol = pt_fCol[1:-4])
    
    f = codecs.open(f"agdSP/agdSP.usp{tb}.Get.sql", mode="w", encoding="utf-8", errors="strict")
    f.write(tmpFile)
    f.close()
    

def GenOneExistSP(tb, dfc):
    dft = df_sp[df_sp["tableName"]==tb]
    tbDscr = dft["tbDscr"].values[0]
    
    tmpFile = codecs.open("template/agdSP.Exist.sql",mode="r",encoding="utf-8",errors='strict', buffering=-1).read().format(tb=tb, tbDscr=tbDscr, pt_DateTime = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
    
    f = codecs.open(f"agdSP/agdSP.usp{tb}.Exists.sql", mode="w", encoding="utf-8", errors="strict")
    f.write(tmpFile)
    f.close()


In [3]:
import pandas as pd
import numpy as np
import pyodbc
import comm as cmm
import time
import codecs

df = pd.read_excel("tableSourceDef.xlsx", sheet_name="admin_columns",converters={'maxLen':str }).replace(np.nan, '')
df_sp = pd.read_excel("tableSourceDef.xlsx", sheet_name="SP_config",converters={'maxLen':str }).replace(np.nan, '')
df["colComp"] = "\t"+df["ColumnName"].apply(cmm.fmtSquare) + (df["Type"]+df["maxLen"].apply(cmm.fmtParentheses)).apply(cmm.fmtSquare) + df["autoGen"]+ df["isNull"].apply(cmm.fmtNull)+","
df["colExt"] = "EXEC sys.sp_addextendedproperty @name=N'MS_Description', @value=N'" + df["ColumnDscr"]+ "' ,@level0type=N'SCHEMA',@level0name=N'" +df["Schema"]+"', @level1type=N'TABLE',@level1name=N'tb"+ df["TableName"]+"', @level2type=N'COLUMN',@level2name=N'"+df["ColumnName"]+"'\nGO"
   
# GenCreateTable(df)
#----------------------------------------------
cnxn = pyodbc.connect(cmm.connMSSQL("newcti"))
query="select SN,tableName,col,fCol,colName,DataType,cmmt_select,cmmt_InputParas,cmmt_Declares,cmmt_SetValue,cmmt_Exec,[sp_Update]  from [dbo].[myTbColumnDscr];"
dfc = pd.read_sql_query(query, cnxn).set_index("SN")
    
tbs = df_sp["tableName"].unique()
for tb in tbs:
    GenOneDeleteSP(tb, dfc)
    GenOneGetSP(tb, dfc)
    GenOneExistSP(tb, dfc)


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [50]:
import pandas as pd
import numpy as np
import pyodbc
import codecs
import comm as cmm





